In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/MSc_AI/Year 1/P5_Data_Mining

/content/drive/MyDrive/MSc_AI/Year 1/P5_Data_Mining


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import torch

In [4]:
df = pd.read_csv("mood_df.csv")

In [5]:
df.columns

Index(['meancircumplex.arousal', 'meancircumplex.valence', 'sumactivity',
       'sumappCat.builtin', 'sumappCat.communication',
       'sumappCat.entertainment', 'sumappCat.finance', 'sumappCat.game',
       'sumappCat.office', 'sumappCat.other', 'sumappCat.social',
       'sumappCat.travel', 'sumappCat.unknown', 'sumappCat.utilities',
       'sumappCat.weather', 'sumcall', 'sumscreen', 'sumsms', 'countactivity',
       'countappCat.builtin', 'countappCat.communication',
       'countappCat.entertainment', 'countappCat.finance', 'countappCat.game',
       'countappCat.office', 'countappCat.other', 'countappCat.social',
       'countappCat.travel', 'countappCat.unknown', 'countappCat.utilities',
       'countappCat.weather', 'countcall', 'countcircumplex.arousal',
       'countcircumplex.valence', 'countscreen', 'countsms', 'id',
       'timestamps', 'y', 'circumplex_rad', 'circumplex_degree'],
      dtype='object')

In [21]:
# A list of all unique people, we can make time series per person.
unique_people = df['id'].unique()
print(unique_people)


ts_idx = df.columns.get_loc('timestamps')
y_idx = df.columns.get_loc('y')

df['timestamps'] = pd.to_datetime(df['timestamps'])

# The amount of consequtive days:
days = 4

X = []
y = []

for person in unique_people:
    one_person_df = df[df['id'] == person]
    #print(one_person_df.columns)
    for row in one_person_df.values:
        n_days = []
        for d in range(days, 0, -1):

            prev_day = row[ts_idx] - pd.DateOffset(days=d)
            if prev_day in one_person_df['timestamps'].values:
                day_df = one_person_df[one_person_df['timestamps'] == prev_day]
                day_df = day_df.drop(columns=['id', 'timestamps', 'y'], axis = 1)
                arrays = day_df.values
                lst = arrays.tolist()
                n_days.append(lst[0])
            #    day_features.append(one_person_df.drop('y', axis=1))
        #n_days_tensor = torch.tensor(n_days)
        X.append(n_days)
        y.append(row[y_idx])



['AS14.01' 'AS14.02' 'AS14.03' 'AS14.05' 'AS14.06' 'AS14.07' 'AS14.08'
 'AS14.09' 'AS14.12' 'AS14.13' 'AS14.14' 'AS14.15' 'AS14.16' 'AS14.17'
 'AS14.19' 'AS14.20' 'AS14.23' 'AS14.24' 'AS14.25' 'AS14.26' 'AS14.27'
 'AS14.28' 'AS14.29' 'AS14.30' 'AS14.31' 'AS14.32' 'AS14.33']


In [22]:
X_new = []
y_new = []

for idx, i in enumerate(X):
    if len(i) ==days:
        X_new.append(i)
        new_y = [0,0,0]
        new_y[int(y[idx])] = 1
        #y_new.append(new_y)
        y_new.append(y[idx])

In [23]:
print(len(X))
print(len(X_new))
print(len(y_new))
print(X_new[0])
print(y_new[0])

1245
1071
1071
[[0.3, 0.0769230769230768, -0.3428848718743589, -0.7763694781866928, -0.3125239211136217, -0.9557345429130516, -0.9605963273593408, -1.0, -1.0, -0.9145335913909204, -0.960882355440416, -0.9558289923078152, -1.0, -0.869318566893948, -1.0, -0.8064516129032258, -0.2025963623672804, -0.8823529411764706, 0.3333333333333332, -0.506631299734748, -0.4037267080745341, -0.9867109634551496, -0.8636363636363636, -1.0, -1.0, -0.7083333333333333, -0.8562874251497006, -0.9736842105263158, -1.0, -0.922077922077922, -1.0, -0.8064516129032258, 0.6666666666666667, 0.3333333333333332, -0.3101604278074866, -0.8823529411764706, -0.3513940678868148, 0.1958794243699566], [0.1, 0.2615384615384615, -0.6287785754309916, -0.6968223898369763, -0.3805408502320461, -0.9552497876824264, -0.9188537861253304, -1.0, -1.0, -0.9365831462851246, -0.919845007170972, -1.0, -1.0, -0.9665733024168416, -0.7996683775221933, -1.0, -0.1205899228178981, -1.0, 0.25, -0.4482758620689655, -0.3975155279503105, -0.9800664

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)


In [25]:
print(unique_people)

['AS14.01' 'AS14.02' 'AS14.03' 'AS14.05' 'AS14.06' 'AS14.07' 'AS14.08'
 'AS14.09' 'AS14.12' 'AS14.13' 'AS14.14' 'AS14.15' 'AS14.16' 'AS14.17'
 'AS14.19' 'AS14.20' 'AS14.23' 'AS14.24' 'AS14.25' 'AS14.26' 'AS14.27'
 'AS14.28' 'AS14.29' 'AS14.30' 'AS14.31' 'AS14.32' 'AS14.33']


In [34]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from imblearn.over_sampling import SMOTE


# Define a custom dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the data and target for the current index
        sample = {'data': self.data[idx], 'target': self.target[idx]}
        return sample

# Assuming you have lists of features and targets for your time series data
# Example: features = [feature_day1, feature_day2, ..., feature_day5]
# Example: targets = [target1, target2, ..., targetN]

# Convert X_train to a numpy array
X_train_array = np.array(X_train)

X_reshaped = []
for xss in X_train_array:
    new_x = []
    for xs in xss:
        for x in xs:
            new_x.append(x)
    X_reshaped.append(new_x)

print(len(X_reshaped))
print(len(y_train))

# Fit and apply the resampling
oversampler = SMOTE()
X_train_resampled_flattened, y_train_resampled = oversampler.fit_resample(X_reshaped, y_train)

# Reshape back
X_train_resampled = []
for xs in X_train_resampled_flattened:
    sub_list = [xs[i:i+len(X_train_array[0][0])] for i in range(0, len(xs), len(X_train_array[0][0]))]
    X_train_resampled.append(sub_list)


print(len(X_train_resampled))
print(len(y_train_resampled))



# Convert lists to numpy arrays
# train_features_array = np.array(X_train_resampled)
# train_targets_array = np.array(y_train_resampled)
train_features_array = np.array(X_train)
train_targets_array = np.array(y_train)
test_features_array = np.array(X_test)
test_targets_array = np.array(y_test)


# Create a dataset instance
train_dataset = TimeSeriesDataset(train_features_array, train_targets_array)
test_dataset = TimeSeriesDataset(test_features_array, test_targets_array)

# Define batch size and shuffle option
batch_size = 1
shuffle = True  # Shuffle the data for training

# Create a DataLoader instance
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)



856
856
1626
1626


In [35]:
for batch in train_loader:
    print(batch['target'])

    break

tensor([1.], dtype=torch.float64)


In [ ]:
import torch
import torch.nn as nn
import tqdm
from sklearn.metrics import classification_report


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Use the output of the last time step
        return out


# Define hyperparameters
input_size = len(X_new[0][0])       # Number of features
hidden_size = len(X_new[0][0])*2     # Number of hidden units in LSTM
num_layers = 4                     # Number of LSTM layers
output_size = 3                     # Output size

# Create an instance of the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


# Train the model
num_epochs = 300
loss = 1
patience = 10
val_loss = 10
best_val_loss = 10
for epoch in range(num_epochs):
    for batch in train_loader:

        inputs = batch['data'].float()  # Convert inputs to float
        targets = batch['target'].long()  # Convert targets to long

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        for batch in train_loader:
            inputs = batch['data'].float()  # Convert inputs to float
            targets = batch['target'].long()
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
    val_loss /= len(test_loader)

    # Check for early stopping

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    # Print validation loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {val_loss:.4f}')

    # Check if early stopping criteria met
    if counter >= patience:
        print(f'Train loss not decreasing for {patience} epochs. Stopping training.')
        break

Epoch [1/300], Train Loss: 3.6627
Epoch [2/300], Train Loss: 3.6108
Epoch [3/300], Train Loss: 3.5956
Epoch [4/300], Train Loss: 3.6308
Epoch [5/300], Train Loss: 3.5629
Epoch [6/300], Train Loss: 3.5529
Epoch [7/300], Train Loss: 3.5428
Epoch [8/300], Train Loss: 3.5776
Epoch [9/300], Train Loss: 3.5112


In [33]:
# Set the model to evaluation mode
model.eval()

# Define a list to store the predictions
predictions = []

# Disable gradient computation
with torch.no_grad():
    # Iterate over the test DataLoader
    for batch in train_loader:
        inputs = batch['data'].float()  # Get input features for the batch
        # Forward pass
        outputs = model(inputs)
        max_values, max_indices = torch.max(outputs, dim=1)
        # Append the predictions to the list
        predictions.append(max_indices)

# Concatenate the predictions
predictions = torch.cat(predictions, dim=0)

# Now you can evaluate the predictions as needed
y_pred = predictions.round().flatten()

print(y_pred)
print(y_test)

report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

tensor([1, 2, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 2, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 1, 2, 0, 0, 1, 1, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1,
        1, 0, 1, 0, 0, 2, 0, 1, 1, 0, 1, 2, 2, 2, 0, 1, 1, 2, 1, 1, 2, 1, 1, 0,
        1, 1, 2, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 2, 2, 1, 1, 2, 0,
        1, 1, 2, 1, 1, 0, 2, 1, 0, 2, 0, 2, 1, 2, 0, 0, 0, 1, 2, 1, 1, 2, 2, 0,
        1, 1, 1, 2, 1, 0, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 2, 1, 1, 1, 1, 2, 1, 2, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1])
[1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 2.0, 2.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0, 1.0, 2